### <font color="red">Import packages </font>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### <font color="red">Read the data into csv file</font>

In [2]:
data =pd.read_csv('data/bike_sharing_data/merged_data.csv')

In [3]:
data.head()

,Duration,Start date,End date,Start station number,Member type
0,892,2018-02-01 00:01:29,2018-02-01 00:16:21,31312,Member
1,501,2018-02-01 00:04:19,2018-02-01 00:12:40,31089,Member
2,788,2018-02-01 00:05:03,2018-02-01 00:18:12,32048,Member
3,406,2018-02-01 00:18:11,2018-02-01 00:24:57,31224,Member
4,161,2018-02-01 00:18:11,2018-02-01 00:20:53,31126,Member


### <font color="red">Select only the prominant station </font>

In [4]:
data_station_31623=data[data['Start station number']==31623] ##this is the data that we are going to use for EDA

In [5]:
data_station_31623=data_station_31623[["Start date","Member type"]]

In [6]:
data_station_31623.head()

,Start date,Member type
60,2018-02-01 05:21:23,Member
100,2018-02-01 05:47:17,Member
101,2018-02-01 05:48:04,Member
121,2018-02-01 06:07:35,Member
145,2018-02-01 06:15:14,Member


In [7]:
data_station_31623.tail()

,Start date,Member type
7147225,2020-02-29 22:07:05,Member
7147289,2020-02-29 22:42:54,Member
7147290,2020-02-29 22:43:54,Member
7147304,2020-02-29 22:56:03,Member
7147366,2020-02-29 23:47:02,Member


In [8]:
data_station_31623["Start date"]=pd.to_datetime(data_station_31623["Start date"],format='%Y-%m-%d %H:%M:%S')

In [9]:
data_station_31623['hour of day']=data_station_31623["Start date"].dt.hour

In [10]:
data_station_31623['day of the month']=data_station_31623["Start date"].dt.day

In [11]:
data_station_31623.head()

,Start date,Member type,hour of day,day of the month
60,2018-02-01 05:21:23,Member,5,1
100,2018-02-01 05:47:17,Member,5,1
101,2018-02-01 05:48:04,Member,5,1
121,2018-02-01 06:07:35,Member,6,1
145,2018-02-01 06:15:14,Member,6,1


In [12]:
data_station_31623['date']=data_station_31623["Start date"].dt.date

In [13]:
count=pd.DataFrame(data_station_31623.groupby(['date','hour of day','Member type'])\
                   ['hour of day'].count().rename('number of users'))

In [14]:
count=count.reset_index()

In [15]:
count.head()

,date,hour of day,Member type,number of users
0,2018-02-01,5,Member,3
1,2018-02-01,6,Member,20
2,2018-02-01,7,Casual,1
3,2018-02-01,7,Member,23
4,2018-02-01,8,Member,24


In [16]:
casual_hourly=count[count['Member type']=="Casual"][['date','hour of day','number of users']]

In [17]:
casual_hourly.head()

,date,hour of day,number of users
2,2018-02-01,7,1
12,2018-02-01,16,1
14,2018-02-01,17,1
17,2018-02-01,19,1
19,2018-02-01,20,1


In [18]:
casual_hourly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4501 entries, 2 to 17835
Data columns (total 3 columns):
date               4501 non-null object
hour of day        4501 non-null int64
number of users    4501 non-null int64
dtypes: int64(2), object(1)
memory usage: 140.7+ KB


In [19]:
casual_hourly.columns

Index(['date', 'hour of day', 'number of users'], dtype='object')

In [20]:
member_hourly=count[count['Member type']=="Member"][['date','hour of day','number of users']]

In [21]:
member_hourly.head()

,date,hour of day,number of users
0,2018-02-01,5,3
1,2018-02-01,6,20
3,2018-02-01,7,23
4,2018-02-01,8,24
5,2018-02-01,9,11


In [22]:
member_hourly=member_hourly.reset_index().drop('index',axis=1)

In [23]:
from datetime import timedelta

In [24]:
member_hourly['date_time']=None
for i in range(len(member_hourly)):
    member_hourly['date_time'][i]=timedelta(hours=int(member_hourly['hour of day'].iloc[i]))+pd.to_datetime(member_hourly['date'].iloc[i])
    

In [25]:
member_hourly=member_hourly[['date_time',"number of users"]]

In [26]:
member_hourly.head()

,date_time,number of users
0,2018-02-01 05:00:00,3
1,2018-02-01 06:00:00,20
2,2018-02-01 07:00:00,23
3,2018-02-01 08:00:00,24
4,2018-02-01 09:00:00,11


In [27]:
casual_hourly=casual_hourly.reset_index().drop('index',axis=1)

In [28]:
casual_hourly.index

RangeIndex(start=0, stop=4501, step=1)

In [29]:
casual_hourly['date_time']=None
for i in range(len(casual_hourly)):
    casual_hourly['date_time'][i]=timedelta(hours=int(casual_hourly['hour of day'].iloc[i]))+pd.to_datetime(casual_hourly['date'].iloc[i])

In [30]:
casual_hourly=casual_hourly[['date_time','number of users']]

In [31]:
casual_hourly.head()

,date_time,number of users
0,2018-02-01 07:00:00,1
1,2018-02-01 16:00:00,1
2,2018-02-01 17:00:00,1
3,2018-02-01 19:00:00,1
4,2018-02-01 20:00:00,1


In [32]:
casual_hourly.shape

(4501, 2)

In [34]:
casual_hourly.to_csv('data/bike_sharing_data/casual_hourly.csv',index=False)

In [35]:
member_hourly.head()

,date_time,number of users
0,2018-02-01 05:00:00,3
1,2018-02-01 06:00:00,20
2,2018-02-01 07:00:00,23
3,2018-02-01 08:00:00,24
4,2018-02-01 09:00:00,11


In [37]:
member_hourly.to_csv('data/bike_sharing_data/member_hourly.csv',index=False)

In [38]:
member_hourly.shape

(13338, 2)

In [39]:
count_by_date=pd.DataFrame(data_station_31623.groupby(['date','Member type'])['hour of day'].count().rename('number of users'))

In [40]:
count_by_date=count_by_date.reset_index()

In [41]:
count_by_date.head()

,date,Member type,number of users
0,2018-02-01,Casual,5
1,2018-02-01,Member,185
2,2018-02-02,Casual,4
3,2018-02-02,Member,113
4,2018-02-03,Casual,1


In [42]:
casual_daily=count_by_date[count_by_date['Member type']=="Casual"][['date','number of users']]

In [43]:
member_daily=count_by_date[count_by_date['Member type']=="Member"][['date','number of users']]

In [44]:
casual_daily.head()

,date,number of users
0,2018-02-01,5
2,2018-02-02,4
4,2018-02-03,1
7,2018-02-05,3
9,2018-02-06,2


In [45]:
casual_daily.shape

(727, 2)

In [47]:
casual_daily.to_csv('data/bike_sharing_data/casual_daily.csv',index=False)

In [48]:
member_daily.head()

,date,number of users
1,2018-02-01,185
3,2018-02-02,113
5,2018-02-03,35
6,2018-02-04,10
8,2018-02-05,163


In [49]:
member_daily.shape

(759, 2)

In [50]:
member_daily.to_csv('data/bike_sharing_data/member_daily.csv',index=False)